### Propensity Model

In [2]:
countryCode = "za"
featureSet = ['density','urbanisation','competition']
featureType =['dum','discrete']
date = '20200903'

In [3]:
dfFeatures = spark.table('data_user_hien.psm_features_' + countryCode)
dfUniverse = spark.table('data_user_hien.psm_universe_' + countryCode)

In [4]:
display(dfFeatures)

operatorID,osmID,PlaceIDGoogle,ohubid,name,address,postalCode,city,latitude,longitude,businessType,website,NameGoogle,AddressGoogle,PostalCodeGoogle,CityGoogle,businessTypeGoogle,WebsiteGoogle,LatitudeGoogle,LongitudeGoogle,type,uid,nameOSM,AddressOSM,PostalCodeOSM,businessTypeOSM,cityOSM,cuisineTypeOSM,websiteOSM,latitudeOSM,longitudeOSM,nameOHUB,addressOHUB,zipCodeOHUB,cityOHUB,concatIdOHUB,channelOHUB,source,globalListChannels,globalChannel,cuisineType,geoHash,ind_popPerHash,urbanisationLevel,ind_businessesPerHash,ind_dum_business_type_An_tent_or_caravan_pitch_location_within_a_campsite_or_caravan_site,ind_dum_business_type_Bakery,ind_dum_business_type_Bar,ind_dum_business_type_Biergarten,ind_dum_business_type_Bowling_alley,ind_dum_business_type_Butcher,ind_dum_business_type_Cafe,ind_dum_business_type_Camp_site,ind_dum_business_type_Caravan_site,ind_dum_business_type_Catering,ind_dum_business_type_Childcare,ind_dum_business_type_Elderly_home,ind_dum_business_type_Fast_food_restaurant,ind_dum_business_type_Food_court,ind_dum_business_type_Food_delivery_restaurant,ind_dum_business_type_Highway_restaurant,ind_dum_business_type_Hospital,ind_dum_business_type_Hostel,ind_dum_business_type_Hotel,ind_dum_business_type_Ice_cream,ind_dum_business_type_Motel,ind_dum_business_type_Movie_theater,ind_dum_business_type_Nursing_home,ind_dum_business_type_Primary_school,ind_dum_business_type_Pub,ind_dum_business_type_Retail,ind_dum_business_type_Schools__Unknown_level,ind_dum_business_type_Secondary_school,ind_dum_business_type_Sports_location,ind_dum_business_type_Stadium,ind_dum_business_type_Student__School_dormitory,ind_dum_business_type_Tertiary_school,ind_dum_business_type_Themeparks,ind_dum_business_type_Visitor_Flats_and_bungalows,ind_dum_business_type_Zoos,ind_dum_urbanisation_1,ind_dum_urbanisation_2,ind_dum_urbanisation_3,ind_dum_urbanisation_4,ind_dum_urbanisation_5,ind_dum_urbanisation_6,ind_dum_urbanisation_7,ind_dum_urbanisation_8,ind_dum_urbanisation_9,dep_dum_buyer
02222a51-234f-4a64-b102-c850fdaa3477,4679075relation,null,02222a51-234f-4a64-b102-c850fdaa3477,Alexandrinenresidenz,Alexandrinenplatz 5,19288,Ludwigslust,53.32396,11.498792,nursing_home,https://www.awo-mv.de/index.php?id=313,null,null,null,null,null,null,null,null,relation,0,Alexandrinenresidenz,Alexandrinenplatz 5,19288,nursing_home,Ludwigslust,null,https://www.awo-mv.de/index.php?id=313,53.32395991349206,11.498791880158732,Alexandrinenresidenz/Awo Kreisverband,Alexandrinenplatz 5,19288,Ludwigslust,DE~BTG~10336903,Alten-/Pflegeheim,OSM,"[Other, Nursing home]",Nursing_home,Unknown,u32pm,10669,10,27,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
03131049-457f-408b-b9be-e03f7a1fd576,null,ChIJfybVpc4epUcRSvE27KNnigc,03131049-457f-408b-b9be-e03f7a1fd576,Parkhotel Weber-Müller,"Sebastian-Kneipp-Promenade 33, 37431 Bad Lauterberg im Harz, Germany",37431,Bad Lauterberg im Harz,51.63284,10.47867,lodging,http://www.harz-parkhotel.de/,Parkhotel Weber-Müller,"Sebastian-Kneipp-Promenade 33, 37431 Bad Lauterberg im Harz, Germany",37431,Bad Lauterberg im Harz,lodging,http://www.harz-parkhotel.de/,51.63284,10.47867,null,null,null,null,null,null,null,null,null,null,null,Parkhotel Weber-Müller GmbH,Sebastian-Kneipp-Promenade 33,37431,Bad lauterberg,DE~EMAKINA~96C01C6C6DCDCEE4A5C05C805D9031D86C328C4C,unknown,GPL,"[Other, Hotel]",Hotel,Unknown,u1pmw,3555,8,28,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1
078c006d-42b8-4769-863d-d6d1122d054b,3870732038,ChIJI2mvbAJ0ukcRl1YAvmd305U,078c006d-42b8-4769-863d-d6d1122d054b,Bistro André,Obermarktstraße,32423,Minden,52.287548,8.916452,restaurant,https://www.bistroandre.de/,Bistro André,"Obermarktstraße 1, 32423 Minden, Germany",32423,Minden,restaurant;food,https://www.bistroandre.de/,52.2874249,8.9164181,node,0,Bistro André,Obermarktstraße 1,32423,restaurant,Minden,null,https://www.bistroandre.de/,52.2875474,8.9164524,Bistro Andre Schaefers Andre,Obermarkt

In [5]:
from scaleai.py_econometrics.glm.fit.fit_probit import fit_probit as fit_probit
from sklearn.model_selection import train_test_split
import pandas as pd
from patsy import dmatrices
import statsmodels.api as sm


/databricks/python/lib/python3.7/site-packages/patsy/constraint.py:13: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
 from collections import Mapping

In [6]:
#set variables

featuresSelection      = featureSet[0] + "_" + featureSet[1]
featuresTypeSelection  = featureType[0] 
idVar =  'operatorID'
depVar = 'dep_dum_buyer'
patternDep = ''
patternInd = 'ind_'
experiment = 'propensity_model_' + countryCode
experimentName = '/Users/hien.ngo@unilever.com/Experiments/' + experiment + '_' + date
artifactLocation = 'dbfs:/user/hien/mlflow/' + experiment + '_' + date
label =  'propensity_model_' + countryCode + '_' + featuresTypeSelection +  '_'+ featuresSelection + '_' + featureSet[2] + '_' +featureType[1] 

In [7]:
print(label)

propensity_model_de_dum_density_urbanisation_competition_discrete

In [8]:
inputColumns = [c for c in dfFeatures.columns if 'dum_density' in c] + [c for c in dfFeatures.columns if 'dum_geo_urbanisation' in c] +['dep_dum_buyer','operatorID'] + ['ind_businessesPerHash']
#ind_popPerHash,ind_businessesPerHash,ind_buildPerHash

dfUniverseWithFeature = dfUniverse.join(dfFeatures,on = ['operatorID'], how ='left_outer')
dfUniverseWithFeature  = dfUniverseWithFeature.select(inputColumns)

dfInputPandas = dfUniverseWithFeature.toPandas()
dfTrain,dfTest = train_test_split(dfInputPandas, test_size = 0.5, random_state =2)



In [9]:
dfScore = fit_probit(
    dfTrain,
    dfInputPandas,
    idVar,
    depVar,
    label,
    patternDep,
    patternInd,
    maxIter = 1000,
    experimentName = experimentName,
    artifactLocation = artifactLocation,
    saveModel = True,
    saveScores = True,
    saveEstimates = True,
    method = 'bfgs')


Optimization terminated successfully.
 Current function value: 0.680223
 Iterations: 72
 Function evaluations: 77
 Gradient evaluations: 77
/local_disk0/spark-9d245402-9374-41a9-8245-0613265b10ce/userFiles-3e7b1de2-e957-405b-853f-f678a86baeb6/addedFile3405379245067731173d292e17e_5baa_41a9_af8b_4f2561e73944_scaleai_1_0_py3_6_3f49f-30c5e.egg/scaleai/py_econometrics/glm/summarize/summarize_probit.py:35: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.

In [10]:
propensityScore = spark.read.option("header", "true") \
                            .option("sep", "|") \
.csv("dbfs:/user/hien/mlflow/propensity_model_za_20200903/577f2db3ae8b4d6ca67afa0225df701b/artifacts/propensity_model_za_dum_density_urbanisation_competition_discrete_dep_dum_buyer_scores.csv") \
#propensity_model_za_20200903/577f2db3ae8b4d6ca67afa0225df701b
propensityScore.count()

Out[2]: 53900

In [11]:
propensityScore.write.mode('overwrite').saveAsTable('data_user_hien.' + label )

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-1551011450927376> in <module> 
 ----> 1 propensityScore . write . mode ( 'overwrite' ) . saveAsTable ( 'data_user_hien.' + label ) 

 NameError : name 'label' is not defined

In [13]:
display(propensityScore)

operatorID,label,score
/ / 0022ade5-2695-4128-b94c-fd068399e75b,dep_dum_buyer,0.42170474999766777
/ / 0043c2db-fbf9-4d99-8f12-2e2734684e28,dep_dum_buyer,0.4629421061803104
/ ChIJ5TDDLy5idkcR3_kiy31tJV8 / 005430b1-02d6-4cdb-b82d-8415cdd905cb,dep_dum_buyer,0.5054784709056622
/ / 006964fa-741f-49ea-908b-16d1521b4895,dep_dum_buyer,0.5259852857384609
/ / 00df637f-8060-4187-a650-ed987da9e76b,dep_dum_buyer,0.486191512410388
/ / 011367b9-6255-42a6-ad22-860bfa04b483,dep_dum_buyer,0.4842141915702735
/ / 0144ba8b-fe60-4e28-a7a6-16e8518d9ec7,dep_dum_buyer,0.5148465361801909
/ ChIJX22qBkCMdkcRB4RcCvywDdk / 0164bf53-4760-3cba-93c8-388a432aafea,dep_dum_buyer,0.4837028601567655
/ / 0171559a-2819-4fa1-a884-26aa1649b938,dep_dum_buyer,0.3859198445153812
/ / 02265e99-0f9e-4e2b-b58d-95b91e746bf7,dep_dum_buyer,0.48495069629504095
